## Load Wikipedia Data

This notebook walks through the process of loading a wikipedia article into a watsonx.data relational database table. We use the [wikipedia python library](https://pypi.org/project/wikipedia/) to retrieve the wikipedia article. We then create a table in the database to store the article. Finally, we load the article into the database.

#### Fetch wikipedia article

Code is provided for searching wikipedia articles as well as fetching a specific article by title.

In [ ]:
import wikipedia

# search
search_results = wikipedia.search("Climate")
search_results

# view article summary
article_summary = wikipedia.summary(search_results[0])
article_summary

In [ ]:
import wikipedia

# fetch wikipedia articles
articles = {
    'Climate change': None, 
    'Climate change mitigation': None
}
for k,v in articles.items():
    article = wikipedia.page(k)
    articles[k] = article.content
    print(f"Successfully fetched {k}")

print(f"Successfully fetched {len(articles)} articles ")

In [ ]:
import wikipedia

wikipedia.set_lang("de")

# fetch wikipedia articles
articles_de = {
    'Globale Erwärmung': None, 
    'Klimaschutz': None
}
for k,v in articles_de.items():
    article_de = wikipedia.page(k)
    articles_de[k] = article_de.content
    print(f"Successfully fetched {k}")

print(f"Successfully fetched {len(articles)} articles ")


In [ ]:
import wikipedia

# search
search_results = wikipedia.search("Climate")
search_results

# view article summary
article_summary = wikipedia.summary(search_results[0])
article_summary

## Load wikipedia article into watsonx.data 

#### Connect to watsonx.data 

In [ ]:
import ssl
import urllib3
import os
from sqlalchemy import create_engine
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # disable https warning

# Because we are connecting to watsonx.data from the Jupyter Server hosted on the same vm as the watsonx.data services 
# Connecting to watsonx.data can be done with the local host credentials listed below:

#these fields do not need to be changed 

LH_HOST_NAME='localhost'
LH_PORT=8443
LH_USER='ibmlhadmin'
LH_PW='password'
LH_CATALOG='tpch'
LH_SCHEMA='tiny'

quick_engine = create_engine(
   f"presto://{LH_USER}:{LH_PW}@{LH_HOST_NAME}:{LH_PORT}/{LH_CATALOG}/{LH_SCHEMA}",
   connect_args={
    'protocol': 'https', 
    'requests_kwargs': {'verify': ssl.CERT_NONE }
    }
)

In [ ]:
## optional connection check (this may take several seconds)
#import pandas as pd 
#df = pd.read_sql('select * from tpch.tiny.customer limit 10', quick_engine)
#df

### Create Schema in watsonx.data Hive Bucket to store wikipedia data

In [ ]:
import pandas as pd

create_schema_result = pd.read_sql("""

CREATE SCHEMA hive_data.watsonxai WITH ( location = 's3a://hive-bucket/watsonx_ai')

""", quick_engine)

# Create table 
create_table_result = pd.read_sql("""

CREATE TABLE hive_data.watsonxai.wikipedia
  (
  "id" varchar,
  "text" varchar,
  "title" varchar  )
WITH (
     format = 'PARQUET'
     )
     
""", quick_engine)

In [ ]:
import pandas as pd

# Create table 
create_table_result = pd.read_sql("""

CREATE TABLE hive_data.watsonxai.wikipedia_de
  (
  "id" varchar,
  "text" varchar,
  "title" varchar  )
WITH (
     format = 'PARQUET'
     )
     
""", quick_engine)

In [ ]:
# Chunk and insert data


# Chunk data
def split_into_chunks(text, chunk_size):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

split_articles = {}
for k,v in articles.items():
    split_articles[k] = split_into_chunks(v, 225)


# Insert data
for article_title, article_chunks in split_articles.items():

    for i, chunk in enumerate(article_chunks):
            
            escaped_chunk = chunk.replace("'", "''").replace("%", "%%")
            insert_stmt = f"insert into hive_data.watsonxai.wikipedia values ('{i+1}', '{escaped_chunk}', '{article_title}')"
            
            with quick_engine.connect() as connection:
                connection.execute(insert_stmt)
            print(f"{article_title} {i+1}/{len(article_chunks)} INSERTED")
            
    print(f"{article_title} DONE")

In [ ]:
# Chunk and insert data


# Chunk data
def split_into_chunks(text, chunk_size):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

split_articles = {}
for k,v in articles_de.items():
    split_articles[k] = split_into_chunks(v, 225)


# Insert data
for article_title, article_chunks in split_articles.items():

    for i, chunk in enumerate(article_chunks):
            
            escaped_chunk = chunk.replace("'", "''").replace("%", "%%")
            insert_stmt = f"insert into hive_data.watsonxai.wikipedia_de values ('{i+1}', '{escaped_chunk}', '{article_title}')"
            
            with quick_engine.connect() as connection:
                connection.execute(insert_stmt)
            print(f"{article_title} {i+1}/{len(article_chunks)} INSERTED")
            
    print(f"{article_title} DONE")

In [ ]:
# confirm data inserted

wiki_articles = pd.read_sql("select * from hive_data.watsonxai.wikipedia", quick_engine)
wiki_articles

In [ ]:
# confirm data inserted

wiki_articles_de = pd.read_sql("select * from hive_data.watsonxai.wikipedia_de", quick_engine)
wiki_articles_de